## Preprocessing for PTV


In [1]:
from pyspark.sql import SparkSession, functions as F

# start a Spark session
spark = SparkSession.builder \
    .appName("PTV Preprocessing") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/17 22:09:04 WARN Utils: Your hostname, WhiteChonk, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/17 22:09:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/17 22:09:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# load the raw geojson file 'public_transport_stops.geojson'
ptv_df = spark.read.json('../data/landing/ptv/public_transport_stops.geojson')

In [9]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

# Read the GeoJSON file as text
with open('../data/landing/ptv/public_transport_stops.geojson', 'r') as f:
    geojson_data = json.load(f)

# Extract features
features = geojson_data['features']

# Create a Spark DataFrame from the features
spark = SparkSession.builder.getOrCreate()
features_df = spark.createDataFrame(features)

# Extract relevant fields into a flat DataFrame
stops_df = features_df.selectExpr(
    "properties.STOP_ID as stop_id",
    "properties.STOP_NAME as stop_name",
    "properties.MODE as mode",
    "geometry.coordinates[0] as longitude",
    "geometry.coordinates[1] as latitude"
    # add more fields if needed
)

# Show a sample of the processed DataFrame
stops_df.show(5)

# Save the processed DataFrame as a Parquet file
stops_df.write.mode('overwrite').parquet('../data/curated/ptv_stops.parquet')

PySparkTypeError: [CANNOT_INFER_TYPE_FOR_FIELD] Unable to infer the type of the field `geometry`.

In [6]:
stops_df = ptv_df.selectExpr("explode(features) as feature") \
    .selectExpr(
        "feature.properties.STOP_NAME as stop_name",
        "feature.properties.STOP_ID as stop_id",
        "feature.properties.MODE as transport_type",
        "feature.geometry.coordinates[0] as lon",
        "feature.geometry.coordinates[1] as lat"
        "feature.geometry.type as geometry_type"
    )


{"ts": "2025-09-17 22:15:40.165", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `features` cannot be resolved. Did you mean one of the following? [`_corrupt_record`]. SQLSTATE: 42703", "context": {"errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o31.selectExpr.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `features` cannot be resolved. Did you mean one of the following? [`_corrupt_record`]. SQLSTATE: 42703; line 1 pos 8;\n'Project ['explode('features) AS feature#17]\n+- Relation [_corrupt_record#6] json\n\n\tat org.apache.spark.sql.errors.QueryCompilationErrors$.unresolvedAttributeError(QueryCompilationErrors.scala:401)\n\tat org.apache.spark.sql.catalyst.analysis.CheckAnalysis.org$apache$spark$sql$catalyst$analysis$Ch

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `features` cannot be resolved. Did you mean one of the following? [`_corrupt_record`]. SQLSTATE: 42703; line 1 pos 8;
'Project ['explode('features) AS feature#17]
+- Relation [_corrupt_record#6] json
